<a href="https://colab.research.google.com/github/ogdura/dinamic/blob/main/Lb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загружаем библиотеки

In [10]:
from re import X
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

Загружаем датасет и подготавливаем данные для обучения

In [5]:
df = pd.read_csv("fraud.csv",sep=',')
df = df.drop('Unnamed: 0', axis=1)
y=df['is_fraud']
x=df.drop(columns=["is_fraud","first","last","trans_date_trans_time", "street", "city", "state","trans_num"], axis=1)
x['dob']=pd.to_datetime(df['dob'], format='%d/%m/%Y')
x['dob']=x['dob'].astype("int64")//10**9
encoder = LabelEncoder()
x['merchant']=encoder.fit_transform(x['merchant'])
x['category']=encoder.fit_transform(x['category'])
x['gender']=encoder.fit_transform(x['gender'])
x['job']=encoder.fit_transform(x['job'])
x

,cc_num,merchant,category,amt,gender,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long
0,2.291160e+15,319,10,2.86,1,29209,33.9659,-80.9355,333497,275,-56419200,1371816865,33.986391,-81.200714
1,3.573030e+15,591,10,29.84,0,84002,40.3207,-110.4360,302,392,632534400,1371816873,39.450498,-109.960431
2,3.598220e+15,611,5,41.28,0,11710,40.6729,-73.5365,34496,259,25315200,1371816893,40.495810,-74.196111
3,3.591920e+15,222,9,60.05,1,32780,28.5697,-80.8191,54767,407,554169600,1371816915,28.812398,-80.883061
4,3.526830e+15,292,13,3.19,1,49632,44.2529,-85.0170,1126,196,-457315200,1371816917,44.959148,-85.884734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,3.056060e+13,507,5,43.77,1,63453,40.4931,-91.8912,519,460,-122515200,1388534347,39.946837,-91.333331
555715,3.556610e+15,264,7,111.84,1,77566,29.0393,-95.4401,28739,198,946252800,1388534349,29.661049,-96.186633
555716,6.011720e+15,496,7,86.88,0,99323,46.1966,-118.9017,3684,294,375840000,1388534355,46.658340,-119.715054
555717,4.079770e+12,75,13,7.99,1,83643,44.6255,-116.4493,129,58,-127699200,1388534364,44.470525,-117.080888


Разделяем данные на обучающиеся и тестируемые

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2,random_state=42)

Создаем классификаторы, добавляем словарь с гиперпараметрами, находим оптимальные гиперпараметры, обучаем и предсказываем классы. После этого можно вывести метрики:

In [12]:
k_range=range(1,11)
knn = KNeighborsClassifier()
kn_params={
    "n_neighbors": k_range,
    'metric': ['minkowski','euclidean','manhattan']
}
kn_grid= GridSearchCV(knn,kn_params,cv=5,n_jobs=1)
kn_grid.fit(x_train,y_train)
best_params=kn_grid.best_params_
knn_best=KNeighborsClassifier(**best_params)
knn_best.fit(x_train,y_train)
pred=knn_best.predict(x_test)
print(best_params)
print(metrics.classification_report(y_test,pred,digits=5))

{'metric': 'manhattan', 'n_neighbors': 1}
              precision    recall  f1-score   support

           0    0.99929   0.99893   0.99911    442833
           1    0.75013   0.81985   0.78344      1743

    accuracy                        0.99822    444576
   macro avg    0.87471   0.90939   0.89128    444576
weighted avg    0.99831   0.99822   0.99826    444576

